In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import f1_score

from sklearn.feature_selection import RFECV
import warnings
warnings.filterwarnings('ignore')

from sklearn.utils import resample

from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
# for feature selection
from sklearn.feature_selection import RFECV
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('cleaned_fire_data.csv')

In [4]:
conditions = [df['stat_cause_descr'] == 'Missing/Undefined',
 df['stat_cause_descr'] == 'Arson',
 df['stat_cause_descr'] == 'Debris Burning',
 df['stat_cause_descr'] == 'Campfire',
 df['stat_cause_descr'] == 'Miscellaneous',
 df['stat_cause_descr'] == 'Fireworks',
 df['stat_cause_descr'] == 'Lightning',
 df['stat_cause_descr'] == 'Equipment Use',
 df['stat_cause_descr'] == 'Children',
 df['stat_cause_descr'] == 'Smoking',
 df['stat_cause_descr'] == 'Railroad',
 df['stat_cause_descr'] == 'Structure',
 df['stat_cause_descr'] == 'Powerline']
outputs = range(0,(len(conditions)))
df['stat_cause_descr'] = np.select(conditions, outputs)

In [5]:
cols_to_drop = [
    'Unnamed: 0',
    'fire_size',
    'fire_size_class',
    'disc_clean_date'
]

In [6]:
df = df.drop(columns=cols_to_drop,axis=1)

In [7]:
df

,stat_cause_descr,latitude,longitude,Temp_pre_30,Temp_pre_15,Temp_pre_7,Wind_pre_30,Wind_pre_15,Wind_pre_7,Hum_pre_30,Hum_pre_15,Hum_pre_7,Prec_pre_30,Prec_pre_15,Prec_pre_7,remoteness,target,month,year,temp_avg,is_remote,did_rain,Temp_pre_30_bin,Temp_pre_15_bin,Temp_pre_7_bin,discovery_month_Aug,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep,state_AL,state_AR,state_AZ,state_CA,state_CO,state_CT,state_DE,state_FL,state_GA,state_HI,state_IA,state_ID,state_IL,state_IN,state_KS,state_KY,state_LA,state_MA,state_MD,state_ME,state_MI,state_MN,state_MO,state_MS,state_MT,state_NC,state_ND,state_NE,state_NH,state_NJ,state_NM,state_NV,state_NY,state_OH,state_OK,state_OR,state_PA,state_PR,state_RI,state_SC,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY,Vegetation_4,Vegetation_9,Vegetation_12,Vegetation_14,Vegetation_15,Vegetation_16,longitude_bin,west_coast,very_windy_30,very_windy_15,very_windy_7,low_humid_30,low_humid_15,low_humid_7
0,0,18.105072,-66.753044,76.065753,76.490462,76.824675,4.341807,3.492857,3.262092,78.216590,76.793750,76.381579,0.0,0.0,0.0,0.017923,0,2,2007,76.460297,0,0,3,3,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3,0,1,1,1,0,0,0
1,1,35.038330,-87.610000,45.596180,44.618000,32.618353,2.709764,2.881707,1.976471,70.840000,65.858911,55.505882,59.8,8.4,0.0,0.184355,0,12,2006,40.944178,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0
2,1,34.947800,-88.722500,40.949474,42.408979,42.005750,3.364499,2.923830,2.695833,75.531629,75.868613,76.812834,168.8,42.2,18.1,0.194544,0,2,2004,41.788067,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0
3,2,39.641400,-119.308300,61.296741,66.193126,64.656615,4.054982,3.398329,3.671282,44.778429,37.140811,35.353846,10.4,7.2,0.0,0.487447,0,6,2005,64.048828,0,1,2,2,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1
4,3,30.904720,-93.557500,62.333490,62.596009,68.782609,1.331257,1.472949,1.424783,72.899478,75.061381,77.924623,28.4,27.5,1.2,0.241894,0,11,2005,64.570703,0,1,2,2,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36530,0,37.606667,-96.422500,38.635038,37.471742,39.987614,5.100510,5.694737,4.975000,62.971774,69.376658,68.118919,20.1,18.8,0.0,0.365622,1,2,2015,38.698132,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0
36531,0,40.394700,-104.564600,67.722291,66.728555,65.620761,2.507911,2.553364,2.638542,51.010341,50.264501,48.204861,4.6,0.0,0.0,0.199532,1,9,2015,66.690536,0,1,2,2,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1
36532,0,39.180000,-96.784167,67.497438,62.404308,66.054190,3.259176,2.705398,3.196648,65.671410,61.839572,54.625698,35.4,8.2,0.0,0.331501,1,10,2015,65.318645,0,1,2,2,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [8]:
# fitting all 3 models
X = df.drop(columns='target',axis=1)
y = df['target']
# train test split + scaler info
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

In [10]:
def LogRegTest(input_x,input_y):
    lr = LogisticRegression(random_state=1,C=1e9)
    lr.fit(input_x,input_y)
    pred_train = lr.predict(input_x)
    score = f1_score(input_y,pred_train,zero_division=1)
    return score

In [11]:
def KnnTest(input_x,input_y):
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(input_x,input_y)
    pred_train = knn.predict(input_x)
    score = f1_score(input_y,pred_train,zero_division=1)
    return score

In [12]:
def DTreeTest(input_x,input_y):
    tree = DecisionTreeClassifier()
    tree.fit(input_x,input_y)
    pred_train = tree.predict(input_x)
    score = f1_score(input_y,pred_train,zero_division=1)
    return score

In [13]:
print('LogReg F1: ', LogRegTest(X_train,y_train))
print('Knn F1: ', KnnTest(X_train,y_train))
print('Dtree F1: ', DTreeTest(X_train,y_train))

LogReg F1:  0.6879693712266236
Knn F1:  0.754874277262337
Dtree F1:  1.0


In [ ]:
# Logistic Regression

lr = LogisticRegression(random_state=2,C=1e9,class_weight='balanced',penalty='l2')

lr.fit(X_train,y_train)

pred_train = lr.predict(X_train)

pred_test = lr.predict(X_test)

score_train = f1_score(y_train,pred_train)
score_test = f1_score(y_test,pred_test)

print('Train: ',score_train)
print('Test: ',score_test)

In [ ]:
j = pd.DataFrame(pred_train)

In [ ]:
j[0].value_counts()

In [ ]:
y_train.value_counts()

In [16]:
# KNN
knn = KNeighborsClassifier(n_neighbors=5,algorithm='auto',weights='uniform')

knn.fit(X_train, y_train)

y_pred_train = knn.predict(X_train)

y_pred_test = knn.predict(X_test)

score_train = f1_score(y_train,pred_train)
score_test = f1_score(y_test,pred_test)

print('Train: ',score_train)
print('Test: ',score_test)

Train:  0.8448459086078641
Test:  0.8071192052980133


In [15]:
#Decision Tree
tree = DecisionTreeClassifier(max_depth=12, min_samples_split=17,criterion='gini',min_samples_leaf=4)

tree = tree.fit(X_train,y_train)

pred_train = tree.predict(X_train)

pred_test = tree.predict(X_test)

score_train = f1_score(y_train,pred_train)
score_test = f1_score(y_test,pred_test)

print('Train: ',score_train)
print('Test: ',score_test)

Train:  0.8448459086078641
Test:  0.8071192052980133


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 3, test_size=0.2)

estimator = LogisticRegression()
# creating a selector object
feature_selector = RFECV(estimator=estimator, step=1, cv=5,n_jobs=-1,min_features_to_select=8)
# fitting to our data
feature_selector.fit(X_train, y_train)

selected_wrapper = X_train.columns[feature_selector.support_]

selected_wrapper

Index(['remoteness', 'is_remote', 'state_AL', 'state_AR', 'state_AZ',
       'state_CA', 'state_CO', 'state_FL', 'state_GA', 'state_ID', 'state_KS',
       'state_KY', 'state_LA', 'state_MS', 'state_MT', 'state_NC', 'state_ND',
       'state_NM', 'state_NY', 'state_OK', 'state_SC', 'state_UT', 'state_WA',
       'state_WI', 'state_WY', 'Vegetation_4', 'Vegetation_9', 'Vegetation_15',
       'longitude_bin', 'west_coast', 'very_windy_30', 'very_windy_15',
       'low_humid_30', 'low_humid_7'],
      dtype='object')

In [21]:
list(selected_wrapper)

['remoteness',
 'is_remote',
 'state_AL',
 'state_AR',
 'state_AZ',
 'state_CA',
 'state_CO',
 'state_FL',
 'state_GA',
 'state_ID',
 'state_KS',
 'state_KY',
 'state_LA',
 'state_MS',
 'state_MT',
 'state_NC',
 'state_ND',
 'state_NM',
 'state_NY',
 'state_OK',
 'state_SC',
 'state_UT',
 'state_WA',
 'state_WI',
 'state_WY',
 'Vegetation_4',
 'Vegetation_9',
 'Vegetation_15',
 'longitude_bin',
 'west_coast',
 'very_windy_30',
 'very_windy_15',
 'low_humid_30',
 'low_humid_7']

# RFE